# Q learning for tic-tac-toe

This was adapted from:
https://towardsdatascience.com/reinforcement-learning-implement-tictactoe-189582bea542

In [1]:
import numpy as np
import pickle
from tqdm import tqdm

BOARD_ROWS = 3
BOARD_COLS = 3

class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1
    
    # get unique hash of current board state
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS*BOARD_ROWS))
        return self.boardHash
    
    def winner(self):
        '''
        Returns:
            1 if p1 wins
            -1 if p2 wins
            0 if draw
            None if not end yet
        '''
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS-i-1] for i in range(BOARD_COLS)])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:
            self.isEnd = True
            return 1
        if diag_sum == -3:
            self.isEnd = True
            return -1
        
        # tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0

        # not end
        self.isEnd = False
        return None
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions
    
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
    
    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)
    
    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
    
    def play_against_computer(self, rounds=100, train=True):
        wins = []

        for i in tqdm(range(rounds)):
            while not self.isEnd:
                # Player 1
                positions = self.availablePositions()
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                # take action and upate board state
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                # check board status if it is end

                win = self.winner()
                if win is not None:
                    wins.append(win)

                    # ended with p1 either win or draw
                    if train:
                        self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)
                    
                    win = self.winner()
                    if win is not None:
                        wins.append(win)

                        # ended with p2 either win or draw
                        if train:
                            self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
        
        print(f"Ratio p1 wins: {wins.count(1)/rounds*100:.2f}%")
        print(f"Ratio p2 wins: {wins.count(-1)/rounds*100:.2f}%")
        print(f"Ratio ties: {wins.count(0)/rounds*100:.2f}%")
    
    # play with human
    def play_against_human(self):
        while not self.isEnd:
            # Player 1
            positions = self.availablePositions()
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
            # take action and upate board state
            self.updateState(p1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.availablePositions()
                p2_action = self.p2.chooseAction(positions)

                self.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == -1:
                        print(self.p2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break

    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------', flush=True)
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out, flush=True)
        print('-------------', flush=True)    

class Player:
    def __init__(self, name, exp_rate=0.3):
        self.name = name
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_value = {}  # state -> value
    
    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS*BOARD_ROWS))
        return boardHash
    
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                if value >= value_max:
                    value_max = value
                    action = p
        return action
    
    # append a hash state
    def addState(self, state):
        self.states.append(state)
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr*(self.decay_gamma*reward - self.states_value[st])
            reward = self.states_value[st]
            
    def reset(self):
        self.states = []
        
    def savePolicy(self):
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

p1 = Player("p1", exp_rate=0.3)
p2 = Player("p2", exp_rate=0.3)

st = State(p1, p2)
print("training...")
st.play_against_computer(100000)

p1.savePolicy()
p2.savePolicy()

training...


100%|██████████| 100000/100000 [02:06<00:00, 792.88it/s]

Ratio p1 wins: 45.32%
Ratio p2 wins: 17.07%
Ratio ties: 37.61%


# Evaluation
https://blog.ostermiller.org/tic-tac-toe-strategy/

To evaluate how well our agent is doing we can evaluate it against a completely random player. According to 
Ostermiller's blog, we can if the agent has become an expert and plays completely optimal moves then the 
rates are the following assuming P1 goes first:

P1 (Expert), P2 (Random):
- P1 wins: 97.8%
- P2 wins: 0.00%
- Ties: 2.20%

P1 (Random), P2 (Expert):
- P1 wins: 0.00%
- P2 wins: 79.6%
- Ties: 20.4%

We can run and evaulation and compare the ratios to see if our agent learned.

In [2]:
p1 = Player("computer", exp_rate=0)
p1.loadPolicy("policy_p1")
p2 = Player("random", exp_rate=1)

st = State(p1, p2)
print("playing...")
st.play_against_computer(10000, train=False)

playing...


100%|██████████| 10000/10000 [00:09<00:00, 1073.41it/s]

Ratio p1 wins: 98.98%
Ratio p2 wins: 0.00%
Ratio ties: 1.02%


In [3]:
p1 = Player("random", exp_rate=1)
p2 = Player("computer", exp_rate=0)
p2.loadPolicy("policy_p2")

st = State(p1, p2)
print("playing...")
st.play_against_computer(10000, train=False)

playing...


100%|██████████| 10000/10000 [00:09<00:00, 1044.62it/s]

Ratio p1 wins: 0.10%
Ratio p2 wins: 57.76%
Ratio ties: 42.14%


We can see that our P1 agent learned the optimal strategy to play because the ratio's match up. However, the P2 agent's ratio does not match. This could be because P1's agent learned the optimal strategy quicker than P2's and thus it couldn't learn anymore.

# Human examples
Here are some examples with a human player:

In [4]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions):
        while True:
            row = int(input("Input your action row:"))-1
            col = int(input("Input your action col:"))-1
            action = (row, col)
            if action in positions:
                return action
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

p1 = Player("computer", exp_rate=0)
p1.loadPolicy("policy_p1")

p2 = HumanPlayer("human")

st = State(p1, p2)
print("Row and Col numbers are from 1 to 3.")
st.play_against_human()

Row and Col numbers are from 1 to 3.
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   |   | o | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   | x | o | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   | x | o | 
-------------
|   | x |   | 
-------------
|   | o |   | 
-------------
-------------
|   | x | o | 
-------------
|   | x | x | 
-------------
|   | o |   | 
-------------
-------------
|   | x | o | 
-------------
| o | x | x | 
-------------
|   | o |   | 
-------------
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
|   | o |   | 
-------------
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
|   | o | o | 
-------------
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
tie!


In [5]:
print("Row and Col numbers are from 1 to 3.")
st.play_against_human()

Row and Col numbers are from 1 to 3.
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   | o | 
-------------
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   | x | o | 
-------------
-------------
|   | o |   | 
-------------
|   | x |   | 
-------------
|   | x | o | 
-------------
-------------
|   | o |   | 
-------------
| x | x |   | 
-------------
|   | x | o | 
-------------
-------------
|   | o |   | 
-------------
| x | x | o | 
-------------
|   | x | o | 
-------------
-------------
|   | o | x | 
-------------
| x | x | o | 
-------------
|   | x | o | 
-------------
-------------
|   | o | x | 
-------------
| x | x | o | 
-------------
| o | x | o | 
-------------
-------------
| x | o | x | 
-------------
| x | x | o | 
-------------
| o | x | o | 
-------------
tie!


In [6]:
print("Row and Col numbers are from 1 to 3.")
st.play_against_human()

Row and Col numbers are from 1 to 3.
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   |   | o | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
|   | x | o | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
| o | x | o | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
| o | x | o | 
-------------
|   | x |   | 
-------------
|   | x |   | 
-------------
computer wins!
